# Before Setting

In [4]:
# 필요 라이브러리 다운로드
import numpy as np
import pandas as pd
import requests
import time
import os
import ssl

from bs4 import BeautifulSoup
from urllib.parse import quote
from selenium import webdriver
from collections import Counter

In [5]:
# 크롬 드라이버 생성
driver = webdriver.Chrome(executable_path = '/Users/hyeonju/Desktop/data-crawling/chromedriver')

# 크롤링 웹사이트 url 설정
url = 'https://search.naver.com/search.naver?where=nexearch&sm=tab_etc&mra=bjBC&qvt=0&query=무료%20전시회'
print(url)

https://search.naver.com/search.naver?where=nexearch&sm=tab_etc&mra=bjBC&qvt=0&query=무료%20전시회


In [6]:
# case 1: requests로 데이터 요청하기
# resp = requests.get(url)

# case 2: SSL 인증 제외 사이트 요청 시
resp = requests.get(url, verify=False)

# 설정한 url 로 데이터 get 함수 요청
driver.get(url)

/Users/hyeonju/opt/anaconda3/envs/mume/lib/python3.8/site-packages/urllib3/connectionpool.py:1045: InsecureRequestWarning: Unverified HTTPS request is being made to host 'search.naver.com'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/1.26.x/advanced-usage.html#ssl-warnings
  warnings.warn(


In [7]:
# # 화면상에서 스크롤 끝까지 내리기
# SCROLL_PAUSE_SEC = 1

# # 스크롤 높이 가져옴
# last_height = driver.execute_script("return document.body.scrollHeight")

# while True:
#     # 끝까지 스크롤 다운
#     driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")

#     # 1초 대기
#     time.sleep(SCROLL_PAUSE_SEC)

#     # 스크롤 다운 후 스크롤 높이 다시 가져옴
#     new_height = driver.execute_script("return document.body.scrollHeight")
#     if new_height == last_height:
#         break
#     last_height = new_height

# # click next page
# # pagination xpath: /html/body/section/div[4]/div/div/form/div[2]/div[2]/ul
#     next_page = driver.find_element_by_xpath()
#     next_page.click()
#     time.sleep(SCROLL_PAUSE_SEC)


In [8]:
# 접근한 페이지의 소스코드를 텍스트로 저장 (Str)
page_text = driver.page_source

# page_text를 html list로 변환 
html = BeautifulSoup(page_text, 'html.parser')

# print(html.prettify(formatter="html5"))

# 네이버 무료전시 url 추출 (리팩토링 필요)

In [9]:
# raw url list
# ✅ Selector: mflick > div > div > div > div > div:nth-child(1) > div.data_area > div > div.title > div > strong > a
# /html/body/div[3]/div[2]/div/div[1]/div[2]/div[2]/div/div/div[3]/div/a[2]
# main_pack > div.sc_new.cs_common_module.case_list.color_1._kgs_art_exhibition > div.cm_content_wrap > div > div > div.cm_paging_area._page > div > span > span._total
url=[]

# 접근한 페이지의 소스코드를 텍스트로 저장 (Str)
# page_text = driver.page_source

# page_text를 html list로 변환
# html = BeautifulSoup(page_text, 'html.parser')

total_page_num = int(html.select_one('div.cm_paging_area._page > div > span > span._total').text)

for i in range(total_page_num):
    page_text = driver.page_source
    html = BeautifulSoup(page_text, 'html.parser')

    for i in html.select('div.area_text_box > strong.this_text > a'):
        urls = i.attrs['href']
        url.append('https://search.naver.com/search.naver' + urls)

    driver.find_element_by_xpath('//*[@id="main_pack"]/div[2]/div[2]/div/div/div[3]/div/a[2]').click()
    time.sleep(1)

print(f"전시회{len(url)}개의 주소입니다.")

print(url)

전시회51개의 주소입니다.
['https://search.naver.com/search.naver?where=nexearch&sm=tab_etc&mra=bjBC&pkid=360&os=29664011&qvt=0&query=%EA%B0%A4%EB%9F%AC%EB%A6%AC%ED%83%9051%EA%B8%B0%EA%B0%90%EC%98%81%EC%A3%BC%EA%B0%9C%EC%9D%B8%EC%A0%84', 'https://search.naver.com/search.naver?where=nexearch&sm=tab_etc&mra=bjBC&pkid=360&os=29664009&qvt=0&query=%EA%B0%A4%EB%9F%AC%EB%A6%AC%ED%83%9051%EA%B8%B0%ED%86%A0%EC%9D%B4%EC%95%84%EC%B9%98%EA%B0%9C%EC%9D%B8%EC%A0%84', 'https://search.naver.com/search.naver?where=nexearch&sm=tab_etc&mra=bjBC&pkid=360&os=29664005&qvt=0&query=%EA%B0%A4%EB%9F%AC%EB%A6%AC%ED%83%9051%EA%B8%B0%EC%9D%B4%EC%9D%B4%EC%98%81%EA%B0%9C%EC%9D%B8%EC%A0%84', 'https://search.naver.com/search.naver?where=nexearch&sm=tab_etc&mra=bjBC&pkid=360&os=29663987&qvt=0&query=%EA%B0%A4%EB%9F%AC%EB%A6%AC%ED%83%9051%EA%B8%B0%EA%B9%80%EC%86%8C%EC%A0%95%EA%B0%9C%EC%9D%B8%EC%A0%84', 'https://search.naver.com/search.naver?where=nexearch&sm=tab_etc&mra=bjBC&pkid=360&os=29565679&qvt=0&query=%ED%82%A4%ED%82%A4%EC%8A

In [10]:
# https://search.naver.com/search.naver 붙여서 url 만들기
# ✅ Selector:

free_exhibition_url = []
for i in url :
  free_exhibition_url.append(f'https://search.naver.com/search.naver{i}')

print(f"네이버 무료 전시 페이지의 {len(free_exhibition_url)}개의 주소입니다")
print(free_exhibition_url)

네이버 무료 전시 페이지의 51개의 주소입니다
['https://search.naver.com/search.naverhttps://search.naver.com/search.naver?where=nexearch&sm=tab_etc&mra=bjBC&pkid=360&os=29664011&qvt=0&query=%EA%B0%A4%EB%9F%AC%EB%A6%AC%ED%83%9051%EA%B8%B0%EA%B0%90%EC%98%81%EC%A3%BC%EA%B0%9C%EC%9D%B8%EC%A0%84', 'https://search.naver.com/search.naverhttps://search.naver.com/search.naver?where=nexearch&sm=tab_etc&mra=bjBC&pkid=360&os=29664009&qvt=0&query=%EA%B0%A4%EB%9F%AC%EB%A6%AC%ED%83%9051%EA%B8%B0%ED%86%A0%EC%9D%B4%EC%95%84%EC%B9%98%EA%B0%9C%EC%9D%B8%EC%A0%84', 'https://search.naver.com/search.naverhttps://search.naver.com/search.naver?where=nexearch&sm=tab_etc&mra=bjBC&pkid=360&os=29664005&qvt=0&query=%EA%B0%A4%EB%9F%AC%EB%A6%AC%ED%83%9051%EA%B8%B0%EC%9D%B4%EC%9D%B4%EC%98%81%EA%B0%9C%EC%9D%B8%EC%A0%84', 'https://search.naver.com/search.naverhttps://search.naver.com/search.naver?where=nexearch&sm=tab_etc&mra=bjBC&pkid=360&os=29663987&qvt=0&query=%EA%B0%A4%EB%9F%AC%EB%A6%AC%ED%83%9051%EA%B8%B0%EA%B9%80%EC%86%8C%EC%A0%95%E

---
# 🛠 데이터 추출 테스트

In [42]:
# Page 1, first card exhibition data crawling

exhibition_url_1_1 = url[45]

request = requests.get(exhibition_url_1_1)

driver.get(exhibition_url_1_1)

In [44]:
# 접근한 페이지의 소스코드를 텍스트로 저장 (Str)
html_text = driver.page_source

# page_text를 html list로 변환 
html = BeautifulSoup(html_text, 'html.parser')

In [45]:
# selector: 
# main_pack > div.sc_new.cs_common_module.case_normal.color_23._kgs_art_exhibition > div.cm_top_wrap._sticky._custom_select._header > div.title_area._title_area > h2 > span.area_text_title > strong > a

# 제목
title_1_1 = html.select_one('div.title_area._title_area > h2 > span.area_text_title > strong > a').text
print(title_1_1)


김용관 : 각진원형


In [50]:
# 장소
# main_pack > div.sc_new.cs_common_module.case_normal.color_23._kgs_art_exhibition > div.cm_content_wrap > div.cm_content_area._cm_content_area_info > div > div.detail_info > dl > div:nth-child(4) > dd > a
# main_pack > div.sc_new.cs_common_module.case_normal.color_23._kgs_art_exhibition > div.cm_content_wrap > div.cm_content_area._cm_content_area_info > div > div.detail_info > dl > div:nth-child(3) > dd > a
# main_pack > div.sc_new.cs_common_module.case_normal.color_23._kgs_art_exhibition > div.cm_content_wrap > div.cm_content_area._cm_content_area_info > div > div.detail_info > dl > div:nth-child(3) > dd > a
place_1_1 = html.select_one('div.sc_new.cs_common_module.case_normal.color_23._kgs_art_exhibition > div.cm_content_wrap > div > div > div.detail_info > dl > div:nth-child(3) > dd > a').text
print(place_1_1)


부산시립미술관


In [15]:
# 날짜
# main_pack > div.sc_new.cs_common_module.case_normal.color_23._kgs_art_exhibition > div.cm_content_wrap > div.cm_content_area._cm_content_area_info > div > div.detail_info > dl > div:nth-child(2) > dd
data_1_1 = html.select_one('dl.info > div:nth-child(2) > dd').text
print(f'전체 기간: {data_1_1}')

startAt_1_1 = data_1_1.split(' ~')[0]
print(f'전시 시작일 : {startAt_1_1}')

endAt_1_1 = data_1_1.split(' ~')[1]
print(f'전시 종료일 : {endAt_1_1}')

전체 기간: 2023.01.04.(수) ~ 2023.04.03.(월)
전시 시작일 : 2023.01.04.(수)
전시 종료일 :  2023.04.03.(월)


In [16]:
# 연령 제한
# 해당 정보 존재하지 않아 pass

In [17]:
# price
# 무료, 별도로 추출하지 않음

In [18]:
# 전시 주최측 제공 url
# main_pack > div.sc_new.cs_common_module.case_normal.color_23._kgs_art_exhibition > div.cm_top_wrap._sticky._custom_select._header > div.title_area._title_area > h2 > span.area_text_title > strong > a
url_1_1 = html.select_one('span.area_text_title > strong > a').attrs['href']
url_1_1


'https://shopping.naver.com/art/stores/100254411/news/5000163495'

# 자동화

### 1. 세부 메서드

### 2. 통합 메서드

In [34]:
def get_crawling_exhibition_info(url):

  print(f'url : {url}')
    
  resp = requests.get(url)
    
  print(f'resp : {resp}')
    
  driver.get (url)

  page_text = driver.page_source

  html = BeautifulSoup(page_text, 'html.parser')

  # get title
  title = html.select_one('div.title_area._title_area > h2 > span.area_text_title > strong > a').text
  
  # get place
  try:
    place = html.select_one('div.sc_new.cs_common_module.case_normal.color_23._kgs_art_exhibition > div.cm_content_wrap > div > div > div.detail_info > dl > div:nth-child(4) > dd > a').text
    
  except:
    place = html.select_one('div.sc_new.cs_common_module.case_normal.color_23._kgs_art_exhibition > div.cm_content_wrap > div > div > div.detail_info > dl > div:nth-child(3) > dd > a').text

  # get starting date, ending date
  try:
    date = html.select_one('div.detail_info > dl.info > div:nth-child(2) > dd').text
    startAt = date.split(' ~')[0]
    endAt = date.split(' ~')[1]
  except:
    startAt = '정보 없음'  
    endAt = '정보 없음'

  # get detail information url
  try: 
    detail_info_url = html.select_one('span.area_text_title > strong > a').attrs['href']
  except: 
    detail_info_url = '정보 없음'
  
  # ----------------------------------------------------------------------------------
  # append
  title_list.append(title)
  place_list.append(place)
  startAt_list.append(startAt)
  endAt_list.append(endAt)
  detail_info_url_list.append(detail_info_url)

# 크롤링 실행

In [35]:
# title
title_list = []

# place
place_list = []

# date
startAt_list = []
endAt_list = []

# detail_info_url_list
detail_info_url_list = []

for element in url :
    get_crawling_exhibition_info(element)
    time.sleep(3)

url : https://search.naver.com/search.naver?where=nexearch&sm=tab_etc&mra=bjBC&pkid=360&os=29664011&qvt=0&query=%EA%B0%A4%EB%9F%AC%EB%A6%AC%ED%83%9051%EA%B8%B0%EA%B0%90%EC%98%81%EC%A3%BC%EA%B0%9C%EC%9D%B8%EC%A0%84
resp : <Response [200]>
url : https://search.naver.com/search.naver?where=nexearch&sm=tab_etc&mra=bjBC&pkid=360&os=29664009&qvt=0&query=%EA%B0%A4%EB%9F%AC%EB%A6%AC%ED%83%9051%EA%B8%B0%ED%86%A0%EC%9D%B4%EC%95%84%EC%B9%98%EA%B0%9C%EC%9D%B8%EC%A0%84
resp : <Response [200]>
url : https://search.naver.com/search.naver?where=nexearch&sm=tab_etc&mra=bjBC&pkid=360&os=29664005&qvt=0&query=%EA%B0%A4%EB%9F%AC%EB%A6%AC%ED%83%9051%EA%B8%B0%EC%9D%B4%EC%9D%B4%EC%98%81%EA%B0%9C%EC%9D%B8%EC%A0%84
resp : <Response [200]>
url : https://search.naver.com/search.naver?where=nexearch&sm=tab_etc&mra=bjBC&pkid=360&os=29663987&qvt=0&query=%EA%B0%A4%EB%9F%AC%EB%A6%AC%ED%83%9051%EA%B8%B0%EA%B9%80%EC%86%8C%EC%A0%95%EA%B0%9C%EC%9D%B8%EC%A0%84
resp : <Response [200]>
url : https://search.naver.com/search.n

# 크롤링 데이터 확인

In [22]:
# Verify title list
title_list

['갤러리탐 51기 : 감영주 개인전',
 '갤러리탐 51기 : 토이아치 개인전',
 '갤러리탐 51기 : 이이영 개인전',
 '갤러리탐 51기 : 김소정 개인전',
 '키키 스미스 ― 자유낙하',
 '강석호 : 3분의 행복',
 '대한민국역사박물관, 시작을 기억하다.',
 '팬데믹 × 디자인 + 미래',
 '우주 백패킹',
 '일인가구',
 '갤러리탐 51기 : 박정애 개인전',
 '갤러리탐 51기 : 양운철 개인전',
 '아트커넥톰',
 '감상일상',
 '어제 본 하루 중에서',
 '지구생존가이드 : 포스트휴먼 2022',
 '예술가의 시그니처',
 '그림은 자연같이',
 '고 이건희 회장 기증 국립광주박물관 특별전',
 '전남미술의 재발견',
 '임선이 : 바람의 무게 - #2 흩어지고 다다른 곳',
 '웰컴 투 조선 - 책 속에 남겨진 조선의 이야기',
 '최민 컬렉션: 다르게 보기',
 '2022 SOMA 공공미술프로젝트',
 'Record: 김종영미술관 20년의 기록',
 '계절산책',
 '다시, 연결 : 모두가 안전해질 때까지',
 '서도호와 아이들: 아트랜드',
 '자연스럼',
 '2030 미래를 향한 꿈 #해시태그 엑스포',
 '수장고 산책: 유리정원',
 '회암사지 그리고 양주',
 '사유정원, 상상 너머를 거닐다',
 '좀비 주의(Attention! Zombies)',
 '7on+',
 '세대공감전',
 '박노해 : 아이들은 놀라워라',
 '주제관 2 - 광고, 세상을 향한 고백',
 '오늘은 어린이날, 소파 방정환의 이야기세상',
 '일본 불교조각의 세계',
 '스마트서울전시관',
 'Hello, Spring',
 '정서윤 개인전',
 '국대호 개인전',
 '송지호 개인전',
 '김용관 : 각진원형',
 '원형정원 프로젝트: 달뿌리-느리고 빠른 대화',
 '김대섭展',
 'Artlee1974 푸른자전거 개인전',
 '박민아 : 수용유희: 콘택트',
 '권민호 : 회색 숨']

In [36]:
# Verify place list
place_list

['탐앤탐스 창동본점',
 '탐앤탐스 블랙그레이트점',
 '탐앤탐스 탐스커버리 건대점',
 '탐앤탐스블랙 압구정점',
 '서울시립미술관',
 '서울시립미술관',
 '대한민국역사박물관',
 '동대문디자인플라자뮤지엄',
 '동대문디자인플라자디자인랩',
 '오산시시립미술관',
 '탐앤탐스 분당 율동공원점',
 '탐앤탐스 블랙 파드점',
 '에이씨티그라운드 화성ICT생활문화센터',
 '김포아트빌리지 아트센터',
 '복합문화공간 111CM',
 '국립아시아문화전당문화창조원',
 '대안공간RGA',
 '대전시립미술관',
 '국립광주박물관',
 '함평군립미술관',
 '우민아트센터',
 '송파책박물관',
 '서울시립미술관',
 '올림픽공원',
 '김종영미술관',
 '중랑아트센터 실감미디어 전시실',
 '대한민국역사박물관',
 '서울시립 북서울미술관',
 '오산시시립미술관',
 '국립부산과학관',
 '국립민속박물관파주',
 '양주시립회암사지박물관',
 '국립아시아문화전당문화창조원',
 '국립아시아문화전당',
 '떼아트 갤러리',
 '아트뮤제 갤러리',
 '라 카페 갤러리',
 '대한민국역사박물관',
 '국립민속박물관 어린이박물관',
 '국립중앙박물관',
 '시민청',
 '갤러리탐 롯데백화점수원점',
 '아트리에 본사',
 '아트리에 본사',
 '아트리에 본사',
 '김용관',
 '국립현대미술관 과천',
 '카페 람',
 '폴어반 르네상스점',
 '강북문화예술회관',
 '권민호']

In [24]:
# Verify start date list
startAt_list

['2023.01.04.(수)',
 '2023.01.04.(수)',
 '2023.01.04.(수)',
 '2023.01.04.(수)',
 '2022.12.15.(목)',
 '2022.12.15.(목)',
 '2022.11.23.(수)',
 '2022.11.15.(화)',
 '2022.11.01.(화)',
 '2023.01.17.(화)',
 '2023.01.04.(수)',
 '2023.01.04.(수)',
 '2022.12.15.(목)',
 '2022.11.23.(수)',
 '2022.11.01.(화)',
 '2022.12.16.(금)',
 '2022.12.16.(금)',
 '2022.11.22.(화)',
 '2022.10.05.(수)',
 '2022.12.20.(화)',
 '2022.11.18.(금)',
 '2023.01.13.(금)',
 '2022.12.08.(목)',
 '2022.10.17.(월)',
 '2022.11.25.(금)',
 '2022.11.09.(수)',
 '2022.09.08.(목)',
 '2022.07.26.(화)',
 '2022.12.13.(화)',
 '2022.09.08.(목)',
 '2022.10.01.(토)',
 '2022.10.20.(목)',
 '2022.12.23.(금)',
 '2022.10.20.(목)',
 '2023.01.17.(화)',
 '2023.01.06.(금)',
 '2022.09.30.(금)',
 '2022.08.16.(화)',
 '2022.05.04.(수)',
 '2022.04.05.(화)',
 '2020.10.27.(화)',
 '2023.01.14.(토)',
 '2023.01.19.(목)',
 '2023.01.19.(목)',
 '2023.01.19.(목)',
 '2022.04.01.(금)',
 '2021.10.08.(금)',
 '2020.11.07.(토)',
 '2022.05.02.(월)',
 '2021.08.01.(일)',
 '2020.10.29.(목)']

In [25]:
# Verify end date list 
endAt_list

[' 2023.04.03.(월)',
 ' 2023.04.03.(월)',
 ' 2023.04.03.(월)',
 ' 2023.04.03.(월)',
 ' 2023.03.12.(일)',
 ' 2023.03.19.(일)',
 ' 2023.02.23.(목)',
 ' 2023.01.31.(화)',
 ' 2023.01.29.(일)',
 ' 2023.04.09.(일)',
 ' 2023.04.03.(월)',
 ' 2023.04.03.(월)',
 ' 2023.02.28.(화)',
 ' 2023.01.29.(일)',
 ' 2023.02.19.(일)',
 ' 2023.02.05.(일)',
 ' 2023.02.26.(일)',
 ' 2023.01.29.(일)',
 ' 2023.01.29.(일)',
 ' 2023.02.19.(일)',
 ' 2023.01.31.(화)',
 ' 2023.08.31.(목)',
 ' 2023.05.07.(일)',
 ' 2023.05.31.(수)',
 ' 2023.03.26.(일)',
 ' 2023.09.16.(토)',
 ' 2023.01.31.(화)',
 ' 2023.03.12.(일)',
 ' 2023.06.25.(일)',
 ' 2023.04.23.(일)',
 ' 2023.02.26.(일)',
 ' 2023.03.26.(일)',
 ' 2023.08.27.(일)',
 ' 2023.03.26.(일)',
 ' 2023.01.30.(월)',
 ' 2023.01.26.(목)',
 ' 2023.10.01.(일)',
 ' 오픈런',
 ' 2024.03.11.(월)',
 ' 2023.10.09.(월)',
 ' 오픈런',
 ' 2023.02.12.(일)',
 ' 2023.02.07.(화)',
 ' 2023.02.07.(화)',
 ' 2023.02.07.(화)',
 ' 2023.04.02.(일)',
 ' 2023.12.17.(일)',
 ' 오픈런',
 ' 2023.12.31.(일)',
 ' 2024.07.31.(수)',
 ' 2023.07.30.(일)']

In [26]:
# Verify detail_info_url list
detail_info_url_list

['https://shopping.naver.com/art/stores/100254411/news/5000163495',
 'https://shopping.naver.com/art/stores/100254411/news/5000163531',
 'https://shopping.naver.com/art/stores/100254411/news/5000163658',
 'https://shopping.naver.com/art/stores/100254411/news/5000163702',
 'https://sema.seoul.go.kr/kr/index',
 'https://sema.seoul.go.kr/kr/index',
 'https://www.much.go.kr/museum/exhibition/exhibitionViewUser.do?exhCode=EXH_0000000169',
 'https://ddp.or.kr/index.html?menuno=230&siteno=2&bbsno=438&boardno=15&cateSched=15&bbstopno=438&act=view',
 'http://www.ddp.or.kr/index.html?menuno=230',
 'https://www.osan.go.kr/arts/program/view.do?mId=0201010000&idx=3283',
 'https://shopping.naver.com/art/stores/100254411/news/5000163299',
 'https://shopping.naver.com/art/stores/100254411/news/5000163424',
 'https://actground.or.kr/exhibition/#(mediaboxes-grid-6194b02aee2a5|filter_1)=.category_category-25;',
 'https://www.gcf.or.kr/modules/rent/event_fest_view.html?mc_code=1013&db_id=1326&page_type=in

# DataFrame

In [38]:
data = {
  'title': title_list,
  'place': place_list,
  'startAt': startAt_list,
  'endAt': endAt_list,
  'detail_info_url': detail_info_url_list 
}

df = pd.DataFrame(data)

df

,title,place,startAt,endAt,detail_info_url
0,갤러리탐 51기 : 감영주 개인전,탐앤탐스 창동본점,2023.01.04.(수),2023.04.03.(월),https://shopping.naver.com/art/stores/10025441...
1,갤러리탐 51기 : 토이아치 개인전,탐앤탐스 블랙그레이트점,2023.01.04.(수),2023.04.03.(월),https://shopping.naver.com/art/stores/10025441...
2,갤러리탐 51기 : 이이영 개인전,탐앤탐스 탐스커버리 건대점,2023.01.04.(수),2023.04.03.(월),https://shopping.naver.com/art/stores/10025441...
3,갤러리탐 51기 : 김소정 개인전,탐앤탐스블랙 압구정점,2023.01.04.(수),2023.04.03.(월),https://shopping.naver.com/art/stores/10025441...
4,키키 스미스 ― 자유낙하,서울시립미술관,2022.12.15.(목),2023.03.12.(일),https://sema.seoul.go.kr/kr/index
5,강석호 : 3분의 행복,서울시립미술관,2022.12.15.(목),2023.03.19.(일),https://sema.seoul.go.kr/kr/index
6,"대한민국역사박물관, 시작을 기억하다.",대한민국역사박물관,2022.11.23.(수),2023.02.23.(목),https://www.much.go.kr/museum/exhibition/exhib...
7,팬데믹 × 디자인 + 미래,동대문디자인플라자뮤지엄,2022.11.15.(화),2023.01.31.(화),https://ddp.or.kr/index.html?menuno=230&siteno...
8,우주 백패킹,동대문디자인플라자디자인랩,2022.11.01.(화),2023.01.29.(일),http://www.ddp.or.kr/index.html?menuno=230
9,일인가구,오산시시립미술관,2023.01.17.(화),2023.04.09.(일),https://www.osan.go.kr/arts/program/view.do?mI...


# Save to .csv File

In [39]:
df.to_csv('not-paid-exhibitions2.csv', index = False, encoding = 'utf-8-sig')